In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from tqdm import tqdm

### v1

In [ ]:
class ReorderRateClassificationModel:
    def __init__(
        self,
        user_id=int,
        orders_df=pd.DataFrame,
        order_products_df=pd.DataFrame,
        top_reodered_items_n: int = 10,
    ):
        self.top_reordered_items_n = top_reodered_items_n

        self._get_orders(orders_df)
        self._get_order_products(order_products_df)

        self.user_id = user_id
        self._get_user_features(self.user_id)

        self._get_products_features()

    def _get_orders(self, orders_df: pd.DataFrame):
        self.orders = orders_df  # меняется на запрос

    def _get_order_products(self, order_products_df: pd.DataFrame):
        self.orders_products = order_products_df  # меняется на запрос

    def _get_last_order_products(self, user_id: int) -> pd.Series:
        prior_orders = self.orders[self.orders["eval_set"] == "prior"]
        prior_user_orders = prior_orders[self.orders["user_id"] == user_id]

        if prior_user_orders.empty:
            return pd.Series([], dtype=int)

        last_prior_user_order_id = prior_user_orders.loc[
            prior_user_orders["order_number"].idxmax(), "order_id"
        ]
        last_order_products = self.orders_products[
            self.orders_products["order_id"] == last_prior_user_order_id
        ]["product_id"]

        return last_order_products

    def _get_top_reordered_items(self) -> pd.Series:
        products_agg = (
            self.orders_products.groupby("product_id")
            .agg(
                {
                    "reordered": "mean",
                }
            )
            .rename(
                columns={
                    "reordered": "product_reorder_rate",
                }
            )
        )
        products_agg.reset_index(inplace=True)

        top_reordered_items = products_agg.sort_values(
            by="product_reorder_rate", ascending=False
        )[: self.top_reordered_items_n]

        return top_reordered_items["product_id"]

    def _get_products_features(self):
        self.df_products_features = pd.DataFrame()

        top_reordered_items = self._get_top_reordered_items().values
        self.df_products_features["top_reordered"] = top_reordered_items

    def _get_user_features(self, user_id: int):
        self.df_user_features = pd.DataFrame()

        last_order_products = self._get_last_order_products(user_id).values
        self.df_user_features["last_user_order"] = last_order_products

    def predict(self, product_id: int, user_id: int = None):
        # Меняется на
        #     features = self.get_features(user_id. product_id)
        #     return lgb_model.predict(features)

        if user_id is not None:
            self.user_id = user_id
            self._get_user_features(self.user_id)

        if product_id in self.df_user_features["last_user_order"].values:
            return True

        if product_id in self.df_products_features["top_reordered"].values:
            return True

        return False

In [ ]:
data_path = "../data/"

orders_df = pd.read_csv(data_path + "orders.csv")
order_products_df = pd.read_csv(data_path + "order_products__prior.csv")

In [ ]:
clf_model = ReorderRateClassificationModel(
    user_id=1, orders_df=orders_df, order_products_df=order_products_df
)

In [ ]:
clf_model.predict(6433)

In [ ]:
class ReorderRateRecommendationModel:
    def __init__(
        self,
        user_id=int,
        orders_df=pd.DataFrame,
        order_products_df=pd.DataFrame,
        top_reodered_items_n: int = 10,
    ):
        self.user_id = user_id

        self.clf_model = ReorderRateClassificationModel(
            user_id, orders_df, order_products_df, top_reodered_items_n
        )

    def predict(self):
        recommendations_list = []
        products_id = self.clf_model.orders["order_id"].values
        for product_id in tqdm(products_id):
            pred = self.clf_model.predict(product_id)
            if pred:
                recommendations_list.append(product_id)

        return np.random.choice(
            recommendations_list, size=clf_model.top_reordered_items_n, replace=False
        )

In [ ]:
rec_model = ReorderRateRecommendationModel(
    user_id=1, orders_df=orders_df, order_products_df=order_products_df
)

In [ ]:
rec_model.predict()


### v2

In [ ]:
data_path = "../data/"

orders_df = pd.read_csv(data_path + "orders.csv")
order_products_prior_df = pd.read_csv(data_path + "order_products__prior.csv")

In [ ]:
print("add order info to priors")
orders_df.set_index("order_id", inplace=True, drop=False)
order_products_prior_df = order_products_prior_df.join(orders_df, on="order_id", rsuffix="_")
order_products_prior_df.drop("order_id_", inplace=True, axis=1)

In [ ]:
order_products_prior_df

In [ ]:
users_priod_products_df = pd.DataFrame()
users_priod_products_df["all_products"] = order_products_prior_df.groupby("user_id")["product_id"].apply(set)

In [ ]:
users_priod_products_df.head()

In [ ]:
train_orders = orders_df[orders_df["eval_set"] == "train"]

In [ ]:
def get_all_pairs(selected_orders, is_train: bool = False):

    order_list = []
    product_list = []
    labels = []

    for row in selected_orders.itertuples():
        order_id = row.order_id
        user_id = row.user_id
        user_products = users_priod_products_df.loc[user_id, "all_products"]
        product_list += user_products
        order_list += [order_id] * len(user_products)

        if is_train:
            labels += [order_products_prior_df.loc[order_id, product]["reordered"] for product in user_products]

    pairs = pd.DataFrame({
        "order_id": order_list,
        "product_id": product_list
    }, dtype=np.int32)

    pairs["user_id"] = pairs.order_id.map(selected_orders.user_id)
    pairs.drop(columns=["order_id"], inplace=True)

    return pairs, labels

In [ ]:
order_products_prior_df.head()

In [ ]:
order_products_prior_df.loc[(order_products_prior_df["order_id"] == 2) & (order_products_prior_df["product_id"] == 33120)]["reordered"]

In [ ]:
pairs = get_all_pairs(train_orders, True)

In [ ]:
pairs

In [ ]:
def get_features(pairs):

    products_agg = (
        order_products_prior_df.groupby("product_id")
        .agg(
            {
                "reordered": "mean",
            }
        )
        .rename(
            columns={
                "reordered": "product_reorder_rate",
            }
        )
    )
    products_agg.reset_index(inplace=True)

    pairs["product_reorder_rate"] = pairs["product_id"].map(
        products_agg.set_index("product_id")["product_reorder_rate"]
    )

    return pairs


In [ ]:
features = get_features(pairs)

In [ ]:
features[features["product_id"] == 196]

In [ ]:
class RecommendationModel:
    def __init__(
        self,
        orders_df: pd.DataFrame,
        order_products_df: pd.DataFrame,
        top_k_products: int = 100,
        recommendations_n: int = 10,
    ):
        self.orders = orders_df
        self.order_products = order_products_df
        self.top_k_products = top_k_products
        self.recommendations_n = recommendations_n

        self.products_agg = (
            self.order_products.groupby("product_id")
            .agg(
                {
                    "reordered": "mean",
                }
            )
            .rename(
                columns={
                    "reordered": "product_reorder_rate",
                }
            )
        )
        self.products_agg.reset_index(inplace=True)

        self.top_reordered_products = (
            self.products_agg.sort_values(by="product_reorder_rate", ascending=False)
            .head(top_k_products)["product_id"]
            .tolist()
        )

    def _get_user_candidates(self, user_id):
        # Получаем последние заказы пользователя
        user_orders = self.orders[self.orders["user_id"] == user_id]
        prior_orders = user_orders[user_orders["eval_set"] == "prior"]

        if prior_orders.empty:
            return self.top_reordered_products

        # Товары из последних заказов пользователя
        last_order_products = self.order_products[
            self.order_products["order_id"].isin(prior_orders["order_id"])
        ]["product_id"].tolist()

        # Комбинируем с топ-товарами
        candidates = list(set(last_order_products + self.top_reordered_products))
        return candidates

    def predict(self, user_id):
        # Получаем список кандидатов для пользователя
        candidates = self._get_user_candidates(user_id)

        # Составляем DataFrame с фичами для кандидатов
        features = self._generate_features(user_id, candidates)

        # Замените self.model.predict(features) на вашу обученную ML-модель
        predictions = np.random.rand(len(features))  # пример случайного предсказания

        # Ранжируем по предсказанным вероятностям и возвращаем топ-N
        top_indices = predictions.argsort()[-self.recommendations_n :][::-1]
        recommended_products = [candidates[i] for i in top_indices]
        return recommended_products

    def _generate_features(self, user_id, candidates):
        # Пример генерации фичей (замените на свои):
        features = pd.DataFrame({"product_id": candidates})
        features["product_reorder_rate"] = features["product_id"].map(
            self.products_agg.set_index("product_id")["product_reorder_rate"]
        )
        # Добавьте больше фичей, если нужно
        return features

In [ ]:
rec_model = RecommendationModel(orders_df, order_products_df)

In [ ]:
rec_model.predict(1)